In [1]:
!pip install nipype

  Using cached nipype-1.8.6-py3-none-any.whl (3.2 MB)
  Using cached simplejson-3.19.2-cp311-cp311-macosx_11_0_arm64.whl (74 kB)
  Using cached traits-6.3.2-cp311-cp311-macosx_10_9_universal2.whl
  Using cached etelemetry-0.3.1-py3-none-any.whl (6.4 kB)
  Using cached looseversion-1.3.0-py2.py3-none-any.whl (8.2 kB)
  Using cached ci_info-0.3.0-py3-none-any.whl (7.8 kB)


In [2]:
# import the relevant Python packages
import numpy
import nibabel
import nipype
import matplotlib
import subprocess

In [4]:
!pip install jedi

In [5]:
!pip install pypet2bids

  Using cached pypet2bids-1.3.9-py3-none-any.whl (199 kB)
  Using cached json_maj-0.0.8-py3-none-any.whl (4.7 kB)
  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached pandas-1.5.3-cp311-cp311-macosx_11_0_arm64.whl (10.8 MB)
  Using cached python_dotenv-0.19.2-py2.py3-none-any.whl (17 kB)
  Using cached pyxlsb-1.0.10-py2.py3-none-any.whl (23 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3


## download Phantom ZIP and extract

In [24]:
!wget -O PHANTOMS.zip https://openneuropet.s3.amazonaws.com/US-sourced-OpenNeuroPET-Phantoms.zip
!unzip PHANTOMS.zip

--2024-05-29 11:22:35--  https://openneuropet.s3.amazonaws.com/US-sourced-OpenNeuroPET-Phantoms.zip
Resolving openneuropet.s3.amazonaws.com (openneuropet.s3.amazonaws.com)... 52.216.44.169, 52.216.220.217, 52.217.71.140, ...
Connecting to openneuropet.s3.amazonaws.com (openneuropet.s3.amazonaws.com)|52.216.44.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115449025 (110M) [application/zip]
Saving to: ‘PHANTOMS.zip’

PHANTOMS.zip        100%[===================>] 110.10M  61.1MB/s    in 1.8s    

2024-05-29 11:22:37 (61.1 MB/s) - ‘PHANTOMS.zip’ saved [115449025/115449025]

Archive:  PHANTOMS.zip
replace __MACOSX/._OpenNeuroPET-Phantoms? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [13]:
import subprocess
check_for_tree = subprocess.run(['tree', '--version'], capture_output=True)
if check_for_tree.returncode == 0:
    pass
else:
    import platform
    operating_system = platform.system()
    if operating_system == 'Linux':
        subprocess.run("apt-get install tree -y", shell=True)
    elif operating_system == 'Darwin':
        subprocess.run("brew install tree", shell=True)
    else:
        print("You're on your own windows user.")

tree is installed


In [23]:
!tree OpenNeuroPET-Phantoms/ --filelimit 15

OpenNeuroPET-Phantoms/
├── README
├── code
│   ├── matlab_conversions.m
│   └── python_conversions.sh
├── dataset_description.json
├── sourcedata
│   ├── GeneralElectricAdvance-NIMH
│   │   ├── 2d_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   │   ├── 3d375_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   │   ├── 3d_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   │   └── long_trans  [35 entries exceeds filelimit, not opening dir]
│   ├── GeneralElectricSignaPETMR-NIMH  [89 entries exceeds filelimit, not opening dir]
│   ├── SiemensBiographPETMR-NIMH
│   │   ├── AC_TOF  [150 entries exceeds filelimit, not opening dir]
│   │   ├── CT  [148 entries exceeds filelimit, not opening dir]
│   │   └── NAC  [148 entries exceeds filelimit, not opening dir]
│   └── SiemensHRRT-JHU
│       └── Hoffman.v
├── sub-GeneralElectricAdvanceLongNIMH
│   └── pet
│       ├── sub-GeneralElectricAdvanceLongNIMH_pet.json
│       └── sub-GeneralElectricAdvanc

In [22]:
# check for dcm2niix
check_dcm2niix = subprocess.run("dcm2niix -h", shell=True, capture_output=True)
if check_dcm2niix.returncode == 0:
    print('dcm2niix is installed')
    # set dcm2niix path as this is running in an ipython notebook
    dcm2niix_path = subprocess.run("which dcm2niix", shell=True, capture_output=True)
    print(dcm2niix_path.stdout.decode())
else:
    print('dcm2niix is not installed')
    import platform
    operating_system = platform.system()

    if operating_system == 'Linux':
        subprocess.run("apt-get install dcm2niix -y", shell=True)
    elif operating_system == 'Darwin':
        subprocess.run("brew install dcm2niix", shell=True)
    else:
        print("You're on your own windows user.")

dcm2niix is installed
/Users/galassiae/fsl/bin/dcm2niix



In [21]:
!dcm2niix4pet ./OpenNeuroPET-Phantoms/source/SiemensBiographPETMR-NRU -d mynewfolder

2024-05-29 11:12:59,264 - pypet2bids - ERROR - Unable to locate dcm2niix executable at None (helper_functions.py:1026)


Traceback (most recent call last):
  File "/Users/galassiae/fsl/bin/dcm2niix4pet", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/Users/galassiae/fsl/lib/python3.11/site-packages/pypet2bids/dcm2niix4pet.py", line 1216, in main
    converter = Dcm2niix4PET(
                ^^^^^^^^^^^^^
  File "/Users/galassiae/fsl/lib/python3.11/site-packages/pypet2bids/dcm2niix4pet.py", line 324, in __init__
    image_header_dict=self.dicom_headers[next(iter(self.dicom_headers))],
                                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
StopIteration


CompletedProcess(args='dcm2niix4pet ./OpenNeuroPET-Phantoms/source/SiemensBiographPETMR-NRU -d mynewfolder', returncode=1)

In [18]:
!tree ./mynewfolder

./mynewfolder
├── Phantom_PetAcquisition_20.01.14-10_39_18-STD-1.3.12.2.1107.5.2.38.51014_20220421120447_30003.json
└── Phantom_PetAcquisition_20.01.14-10_39_18-STD-1.3.12.2.1107.5.2.38.51014_20220421120447_30003.nii.gz

0 directories, 2 files


In [19]:
!dcm2niix4pet ./OpenNeuroPET-Demo_raw/source/SiemensBiographPETMR-NRU -d mynewfolder2 --kwargs TimeZero=ScanStart Manufacturer=Siemens ManufacturersModelName=Biograph InstitutionName="Rigshospitalet, NRU, DK" BodyPart=Phantom Units=Bq/mL TracerName=none TracerRadionuclide=F18 InjectedRadioactivity=81.24 SpecificRadioactivity=13019.23 ModeOfAdministration=infusion FrameTimesStart=0 AcquisitionMode="list mode" ImageDecayCorrected=true ImageDecayCorrectionTime=0 AttenuationCorrection=MR-corrected FrameDuration=300 FrameTimesStart=0

Attempting to locate missing BIDS fields in dicom header
FOUND BodyPartExamined corresponding to BIDS BodyPart: BRAIN
NOT FOUND MappingResourceName corresponding to BIDS TracerName in dicom header.
NOT FOUND CodeMeaning corresponding to BIDS TracerRadionuclide in dicom header.
NOT FOUND RadionuclideTotalDose corresponding to BIDS InjectedRadioactivity in dicom header.
NOT FOUND RadiopharmaceuticalSpecificActivity corresponding to BIDS MolarActivity in dicom header.
NOT FOUND RadiopharmaceuticalVolume corresponding to BIDS InjectedVolume in dicom header.
NOT FOUND InterventionDrugName corresponding to BIDS PharmaceuticalName in dicom header.
NOT FOUND InterventionDrugDose corresponding to BIDS PharmaceuticalDoseAmount in dicom header.
NOT FOUND RadiopharmaceuticalStartTime corresponding to BIDS InjectionStart in dicom header.
FOUND AcquisitionDate corresponding to BIDS ScanDate: 20220421
NOT FOUND RadiopharmaceuticalStopTime corresponding to BIDS InjectionEnd in dicom header.
FOUND Reco

## validate BIDS dataset
https://www.npmjs.com/package/bids-validator

In [ ]:
!pip install bids_validator

In [ ]:
from bids_validator import BIDSValidator

In [ ]:
BIDSValidator().is_bids('.') # not up to date, use browser version for now

## run only if freesurfer is installed

In [ ]:
!mkdir PETprocessing

In [ ]:
!cd PETprocessing

In [ ]:
!git clone https://github.com/openneuropet/PET_pipelines.git 

In [ ]:
!cd PET_pipelines/pyPetSurfer 

In [ ]:
!wget https://www.dropbox.com/sh/69dwtnv29wd7jlx/AADnw5FvAANpvzKAxVQTnyhBa?dl=0 

In [ ]:
!ls PET_pipelines/pyPetSurfer

In [ ]:
!unzip AADnw5FvAANpvzKAxVQTnyhBa?dl=0

In [ ]:
!pip install mri_coreg

In [ ]:
%run ./example.py

In [ ]:
!ls .
